# T2.2: PANDAS. Ler excels de alcaldes



<p>Instalación de paquetes necesarios</p>

In [ ]:
%conda install pandas # o  %pip install pandas
%conda install datetime #o %pip install datetime
%conda install unidecode # o %pip install unidecode

<p>Importar librerías</p>

In [19]:
import pandas as pd
from datetime import datetime
from unidecode import unidecode

Normalización del data frame

In [ ]:
excel_entero = pd.read_excel("Alcades_1979_2023.xlsx", na_values='\\N', sheet_name=None)


def limpiar(df): 
    df = df.dropna(axis=1, how='all') # Quitar valores NaN
    df = df.drop([0,1,2,3,4,5,6]) # Quitar columnas innecesarias
    df = df.drop(['Unnamed: 13'], axis=1) # Quitar columna vacía
    df.rename(columns={'Unnamed: 3': 'COMUNIDAD AUTONOMA', 'Unnamed: 4': 'PROVINCIA', 'Unnamed: 5': 'MUNICIPIO', 'Unnamed: 6': 'CODIGO INE', 'Unnamed: 7': 'NOMBRE', 'Unnamed: 8': 'LISTA', 'Unnamed: 9': 'FECHA POSESION', 'Unnamed: 10': 'FECHA BAJA'}, inplace=True) # Renombrar header
    df = df.map(lambda s: unidecode(s.lower()) if isinstance(s, str) else s) # Quitar acentos y convertirlo a minuscula

    for fecha_col in ['FECHA POSESION', 'FECHA BAJA']: # Cambiar formato de la fecha 
        if fecha_col in df.columns:
            df[fecha_col] = pd.to_datetime(df[fecha_col], errors='coerce',dayfirst=True).dt.strftime('%d/%m/%Y') 


    return df 

excel_entero_df = pd.concat([limpiar(excel_entero['Alcaldes 1979-1983']),limpiar(excel_entero['Alcaldes 1983-1987']),limpiar(excel_entero['Alcaldes 1987-1991']),limpiar(excel_entero['Alcaldes 1991-1995']),limpiar(excel_entero['Alcaldes 1995-1999']),limpiar(excel_entero['Alcaldes 1999-2003']),limpiar(excel_entero['Alcaldes 2003-2007']),limpiar(excel_entero['Alcaldes 2007-2011']),limpiar(excel_entero['Alcaldes 2011-2015']),limpiar(excel_entero['Alcaldes 2015-2019']),limpiar(excel_entero['Alcaldes 2019-2023'])])


excel_entero_df.head(50)




### Fai algunhas averiguacións sobre os datos:

<p>1. Saber qué alcaldes repetiron mandato e en qué lexislaturas (ollo, poden mudar un pouco os nomes).</p>

<p>2. De repetir mandato. Qué partidos políticos tiveron máis éxito? Qué alcaldes mudaron de partido?</p>

<p>3. Cantos alcaldes cesaron antes de tempo?</p>

<p>4. Cal é o soldo máis alto e de que alcaldes?</p>

<p>5. Relaciona os soldos altos e baixos coa "probabilidade" de repetir mandato.</p>

<p>6. Outra información interesante.</p>
